# Approximate Nearest Neighbours Search

Sometimes, when we are processing a user query, it may be **acceptable to retrieve a "good guess"** of 
nearest neighbours to the query **instead of true nearest neighbours**. In those cases, one can use an algorithm which doesn't guarantee to return the actual nearest neighbour in every case, **in return for improved speed or memory savings**. Thus, with the help of such algorithms one can do a **fast approximate search in a very large dataset**. Today we will expore one of such approaches based on graphs.

This is what we are going to do in this lab: 

1. Complete implementation of small-world graph;
2. Implement search in this graph;
3. Build a *navigable* small-world graph;
4. Compare search quality in the resulting graphs.


## 1. Dataset preparation
We will utilize the same dataset which was used in the last lab - the [dataset with curious facts](https://github.com/hsu-ai-course/hsu.ai/blob/master/code/datasets/nlp/facts.txt). Using trained `doc2vec` [model](https://github.com/jhlau/doc2vec) (Associated Press News DBOW (0.6GB), we will infer vectors for every fact and normalize them.


### 1.1 Loading doc2vec model

In [2]:
!pip install gensim
!wget https://public.boxcloud.com/d/1/b1!d5q3hz-n1_jkOcrgoeBV8-5_4U4IgAOfPgCfSASzlzUTH7D4W5znMtd0NfiO8ORZRcY50xfC5XKtOZ002MsRcBDrb28D_KLRI3p-OV12Wi0lr7UfQqDElXqIt_4pSLfU6LSZcAN1xnYq2k1DtIVNqGQ6z5tYvcsYVr-yS4mxrREyJfFLNYd3JShiSA5WL7rpd_80FR0ZoyLZhW95yOF80aVniiXaPv5_aj557hf0g-aPA-S1iQpdTO_fGsZihKVBqK3fbi0F-u9iUr6JWN8MU25gONXOsl1BiZiU1dvAWXo-caqeXkNWtB4Cz4NJGb7aUjFuGUq4PaW9JRTRjKbbRUJNB_alhnMnRIJeiLv8p9-bPRbpg0dciy4LVxYc2zqKCPQ9w5AnvUgqM8Tm0fKRRH-pvyJYKK_cGAZ56xnR2CK8V9boLT9LnSq1ykTbnzSPJfZbN5sMX0_MBmXHxadUfrH-VZefNKym--qW8VCk9nVIdxYhkfpOaL4Q8I4__Il5wT7jVuvIWf_7PkmCEdNoZwCoxPtJji_A18DXoiJdbKX5sBJHkF0sukWW0mMLSrdjf2ZDXXwwZHOaJskvfBTTK_uPUQZ-cBIUeZ8UXt6AlTZfhM_dli2uxDmKiJIxVqfAZHXhIwFcW9GtPysscp-CDxiJ-fERcpOTdaA2Mo9bkB90PQgyOq1ZLqqCQ7jwJLXI6ooWxb0tWRpgd8SpXDr72k3dT32eJJYiWrkWeNiZd1LcFH2EY76mMMPqaNv2J4Q0O9aWjCnzmgjXySnOJZde1Glt8iTdHALakS77o8P3tR-KiAijy7yec6K6nu9Lv3X5z1Npq3dkY2W6BlFCSB0i7S7VdIBMD6qBaD6ir_-pwFJTKCDRYHZSD2GVsl7kBIu5DdCiZf-drDGzEhMLNZ93xtby-1fQxIJnHpfmNlKCA9bzI5UYL5Pf5hm-nCaESKijcbiD2qPbdHqsFm3XpbfEJmCTpMwYMF0LEAy2FfgxHnOVzO9D1NluiacdcIXKDsnvOINPjtEhePKEaCFNfZp2ovmJtTxhnX6ynKDomW5MCqBi0QTbV_s_lm-uV9r-WiS3QT3nUEQzf25t3xgU5yNxU79nPJqXusXIifTnTpSfjcoNg62XDDu16VKmw34lHHHiWmjJrfupgpqCMw1XY9U2HSWPPmP7U8lCL7h43SMX6CJpOa4BM2yV7SNzAsNpOgRkwtOfsq90qddE6RWJpw9YQjGNzBKOctFuttTdEJIJNd_rclRJ4kqwAvE0G47AFHFNV2xyWeGtxV49pLqqM_Uk7NvzVefhizRbXTJhyzo1WTlJxgL2Ebbj2LACdctL2KQ./download

--2020-02-23 18:12:21--  https://public.boxcloud.com/d/1/b1!d5q3hz-n1_jkOcrgoeBV8-5_4U4IgAOfPgCfSASzlzUTH7D4W5znMtd0NfiO8ORZRcY50xfC5XKtOZ002MsRcBDrb28D_KLRI3p-OV12Wi0lr7UfQqDElXqIt_4pSLfU6LSZcAN1xnYq2k1DtIVNqGQ6z5tYvcsYVr-yS4mxrREyJfFLNYd3JShiSA5WL7rpd_80FR0ZoyLZhW95yOF80aVniiXaPv5_aj557hf0g-aPA-S1iQpdTO_fGsZihKVBqK3fbi0F-u9iUr6JWN8MU25gONXOsl1BiZiU1dvAWXo-caqeXkNWtB4Cz4NJGb7aUjFuGUq4PaW9JRTRjKbbRUJNB_alhnMnRIJeiLv8p9-bPRbpg0dciy4LVxYc2zqKCPQ9w5AnvUgqM8Tm0fKRRH-pvyJYKK_cGAZ56xnR2CK8V9boLT9LnSq1ykTbnzSPJfZbN5sMX0_MBmXHxadUfrH-VZefNKym--qW8VCk9nVIdxYhkfpOaL4Q8I4__Il5wT7jVuvIWf_7PkmCEdNoZwCoxPtJji_A18DXoiJdbKX5sBJHkF0sukWW0mMLSrdjf2ZDXXwwZHOaJskvfBTTK_uPUQZ-cBIUeZ8UXt6AlTZfhM_dli2uxDmKiJIxVqfAZHXhIwFcW9GtPysscp-CDxiJ-fERcpOTdaA2Mo9bkB90PQgyOq1ZLqqCQ7jwJLXI6ooWxb0tWRpgd8SpXDr72k3dT32eJJYiWrkWeNiZd1LcFH2EY76mMMPqaNv2J4Q0O9aWjCnzmgjXySnOJZde1Glt8iTdHALakS77o8P3tR-KiAijy7yec6K6nu9Lv3X5z1Npq3dkY2W6BlFCSB0i7S7VdIBMD6qBaD6ir_-pwFJTKCDRYHZSD2GVsl7kBIu5DdCiZf-drDGzEhMLNZ93xtby-1fQxIJnHpfmNlKCA9bz

In [3]:
from gensim.models.doc2vec import Doc2Vec
! tar -xvf download && mv apnews_dbow/*

# unpack a model into 3 files and target the main one
# doc2vec.bin  <---------- this
# doc2vec.bin.syn0.npy
# doc2vec.bin.sin1neg.npy





model = Doc2Vec.load('apnews_dbow/doc2vec.bin', mmap=None)
print(type(model))
print(type(model.infer_vector(["to", "be", "or", "not"])))

apnews_dbow/
apnews_dbow/doc2vec.bin.syn1neg.npy
apnews_dbow/doc2vec.bin.syn0.npy
apnews_dbow/doc2vec.bin
mv: target 'apnews_dbow/doc2vec.bin.syn1neg.npy' is not a directory


/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:402: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


<class 'gensim.models.doc2vec.Doc2Vec'>
<class 'numpy.ndarray'>


/usr/local/lib/python3.6/dist-packages/gensim/models/doc2vec.py:566: UserWarning: The parameter `iter` is deprecated, will be removed in 4.0.0, use `epochs` instead.
  warnings.warn("The parameter `iter` is deprecated, will be removed in 4.0.0, use `epochs` instead.")
/usr/local/lib/python3.6/dist-packages/gensim/models/doc2vec.py:570: UserWarning: The parameter `size` is deprecated, will be removed in 4.0.0, use `vector_size` instead.
  warnings.warn("The parameter `size` is deprecated, will be removed in 4.0.0, use `vector_size` instead.")


### 1.2 Reading data

In [0]:
import urllib.request
data_url = "https://raw.githubusercontent.com/hsu-ai-course/hsu.ai/master/code/datasets/nlp/facts.txt"
file_name= "facts.txt"
urllib.request.urlretrieve(data_url, file_name)



facts = []
with open(file_name,encoding= "windows-1251") as fp:
    for cnt, line in enumerate(fp):
        facts.append(line.strip('\n'))

### 1.3 Transforming sentences into vectors

In [5]:
import nltk
nltk.download('punkt')
import numpy as np

def word_tokenize(sentence):
    return nltk.word_tokenize(sentence.lower())

def get_words_from_sentence(sentences):
    for sentence in sentences: 
        yield word_tokenize(sentence.split('.', 1)[1])

sent_vecs = np.array([])
sent_vecs = np.array(list(model.infer_vector(words) for words in get_words_from_sentence(facts)))

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


### 1.4 Normalizing vectors

In [0]:
def norm_vectors(A):
    An = A.copy()
    for i, row in enumerate(An):
        An[i, :] /= np.linalg.norm(row)
    return An

def find_k_closest(query, dataset, k=5):    
    index = list((i, v, np.dot(query, v)) for i, v in enumerate(dataset))    
    return sorted(index, key=lambda pair: pair[2], reverse=True)[:k]

sent_vecs_normed = norm_vectors(sent_vecs)

## 2. Small world network ##
We discussed [small world networks](https://en.wikipedia.org/wiki/Small-world_network) in lecture. This beautiful concept utilizes skip-list idea to reach query neighbourhood fastly from any random graph node. You have practically all code written, you just need to complete `rewire()` function with respect to [Watts–Strogatz process](https://en.wikipedia.org/wiki/Watts%E2%80%93Strogatz_model).

**Please write rewiring code.**

Function `build_graph` accepts some iterable collection of `values`. In our case this will be embeddings. 

- `K` is a parameter of Watts–Strogatz model, expressing average degree of graph nodes.
- `p` stands for probability of "rewiring".

In [0]:
import random
class Node:
    ''' Graph node class. Major properties are `value` to access embedding and `neighbourhood` for adjacent nodes '''
    def __init__(self, value, idx):
        self.value = value
        self.idx = idx
        self.neighbourhood = set()
        

def build_graph(values, K, p=0.4):
    '''Accepts container with values. Returns list with graph nodes'''
    
    def rewire(nodes, i, j, k):
        #TODO remove i-j connection and add i-k connection, bi-directional
        nodes[i].neighbourhood.discard(j %N)
        nodes[j %N].neighbourhood.discard(i)
        nodes[i].neighbourhood.add(k)
        nodes[k].neighbourhood.add(i)
        return True
    
        
    N = len(values)
    nodes = [None] * N
    
    # create nodes
    for i, val in enumerate(values):
        nodes[i] = Node(val, i)
    
    # create K-regular lattice
    for i, val in enumerate(nodes):
        for j in range(i - K // 2, i + K // 2 + 1):
            if i != j:
                nodes[i].neighbourhood.add(j % N)
                nodes[j % N].neighbourhood.add(i)
        
    for i, node in enumerate(nodes):
      #TODO for each node rewire right hand side i-j edge to some other random node
      # See Watts–Strogatz model for details
      for ko in range (i+1,i + K // 2 + 1):
        if random.random()<=p:
          potential = None
          while potential == None or potential in nodes[i].neighbourhood : 
            potential = random.randint(0,len(nodes)-1)
          rewire(nodes,i,ko,potential)

      pass
                
    return nodes

The bigger `K` and `p` you choose, the longer method runs. Bigger `K` leads to bigger near-cliques in a graph and, as a consequence, bigger context to consider at each step of search. Bigger `p` is for a better "remote hops", but it should not be close to 1, as it will make graph random (not SW).

In [14]:
import time
start = time.time()
G = build_graph(sent_vecs_normed, K=10, p=0.2)
finish = time.time()
print("Graph built in {:.2f} ms".format(1000 * (finish - start)))

Graph built in 1.57 ms


### 2.1 Searching in a small-world graph

Now you need to implement an efficient search procedure which would utilize small world properties. Starting from the random node, at each step you should move towards the closest node (in terms of cosine simiarity, in our case), meanwhile keeping and refreshing top-K nearest neightbours collection. 

**Please implement basic NSW search**. 

You can refer to the `K-NNSearch` algorithm which pseudocode is given in section 4.2 of the [original paper](https://publications.hse.ru/mirror/pubs/share/folder/x5p6h7thif/direct/128296059).

`search_nsw_basic()`
- `query` - `vector` (`np.ndarray`) representing a query.
- `nsw` - SW graph.
- `top` - re-ranking set size.
- `guard_hops` - if method does not converge, we will terminate when reaching guard_hops #steps.
- `returns` - a pair of a `set` of indices and number of hops `(nearest_neighbours_set, hops)`

In [0]:
import sortedcontainers
from scipy.spatial import distance

def search_nsw_basic(query, nsw, top=5, guard_hops=100):
    ''' basic search algorithm, takes vector query and returns a pair (nearest_neighbours, hops)'''
    #TODO implement basic NSW search
    hops = 0    
    c_poss = nsw[random.randint(0,len(nsw)-1)]
    c_curr = 1
    cos_best = 0.999
    index_b = nsw[random.randint(0,len(nsw)-1)]
    nearest_neighbours_set = []
    while c_curr > cos_best:
      c_curr = cos_best
      c_poss = index_b
      nearest_neighbours_set.append(c_poss.idx)
      for b in (c_poss.neighbourhood):
        
        if distance.cosine(query, nsw[b].value)<cos_best:
          cos_best, index_b = distance.cosine(query, nsw[b].value),nsw[b]
      hops+=1
      if hops == 100:
        break

  
    return nearest_neighbours_set, hops

### 2.2 Test the search procedure

In [0]:
test_queries = ["good mood", "birds", "virus and bacteria"]
test_vectors = np.array([model.infer_vector(word_tokenize(q)) for q in test_queries])
test_queries_normed = norm_vectors(test_vectors)

First, let's display the true nearest neighbours and measure average search time. 

In [17]:
search_time = 0
for i, query in enumerate(test_queries):
    start = time.time()
    r = find_k_closest(test_queries_normed[i], sent_vecs_normed)
    finish = time.time()
    search_time += finish - start  

    print("\nResults for query:", query)
    for k, v, p in r:
        print("\t", facts[k], "sim=", p)

print("\nExact search took {:.4f} ms on average".format(1000 * (search_time/len(test_queries))))


Results for query: good mood
	 68. Cherophobia is the fear of fun. sim= 0.6169215
	 45. About half of all Americans are on a diet on any given day. sim= 0.54591876
	 76. You breathe on average about 8,409,600 times a year sim= 0.5296933
	 144. Dolphins sleep with one eye open! sim= 0.52570844
	 57. Gorillas burp when they are happy sim= 0.5225472

Results for query: birds
	 47. Avocados are poisonous to birds. sim= 0.7113577
	 121. Birds don’t urinate. sim= 0.65965736
	 111. Butterflies taste their food with their feet. sim= 0.65335965
	 109. Cows kill more people than sharks do. sim= 0.640721
	 73. Scientists have tracked butterflies that travel over 3,000 miles. sim= 0.59202904

Results for query: virus and bacteria
	 39. A 2010 study found that 48% of soda fountain contained fecal bacteria, and 11% contained E. Coli. sim= 0.6250343
	 47. Avocados are poisonous to birds. sim= 0.6076415
	 109. Cows kill more people than sharks do. sim= 0.6055639
	 54. Coconut water can be used as blo

Now, let's see `search_nsw_basic` in action. It should work way faster than pairwise comparisons above.

In [18]:
search_time = 0
for i, query in enumerate(test_queries):
    start = time.time()
    ans, hops = search_nsw_basic(test_queries_normed[i], G)
    finish = time.time()
    search_time += finish - start

    print("\nResults for query:", query)
    for k in ans:
        print("\t", facts[k], "sim=", np.dot(test_queries_normed[i], sent_vecs_normed[k]))
        
print("\nBasic nsw search took {:.4f} ms on average".format(1000 * (search_time/len(test_queries))))    


Results for query: good mood
	 116. Male dogs lift their legs when they are urinating for a reason. They are trying to leave their mark higher so that it gives off the message that they are tall and intimidating. sim= 0.4144693
	 54. Coconut water can be used as blood plasma. sim= 0.45134032
	 57. Gorillas burp when they are happy sim= 0.5225472

Results for query: birds
	 142. The placement of a donkey’s eyes in its’ heads enables it to see all four feet at all times! sim= 0.4785607
	 144. Dolphins sleep with one eye open! sim= 0.58778775

Results for query: virus and bacteria
	 81. Under the Code of Hammurabi, bartenders who watered down beer were punished by execution. sim= 0.3394858
	 83. During your lifetime, you will produce enough saliva to fill two swimming pools. sim= 0.5822532

Basic nsw search took 2.9117 ms on average


The results you see should be worse than the exact nearest neighbours, however, not completely random. Pay attention to the similarity values.

## 3. Navigable small-world graph

When building small-world graph using Watts–Strogatz model, there was no notion of proximity between the nodes - it was completely ignored. In Navigable small-world graphs, however, the idea is to insert nodes in such a way that the cliques form real neighbourhoods, meaning points that are connected are close to each other. Please refer to section 5 of the [paper](https://publications.hse.ru/mirror/pubs/share/folder/x5p6h7thif/direct/128296059) for the details.

In [0]:
def build_navigable_graph(values, K):
    '''Accepts container with values. Returns list with graph nodes.
    K parameter stands for the size of the set of closest neighbors to connect to when adding a node'''
    #TODO implement navigable small-world graph consrtuction  
       
    N = len(values)
    nodes = [None] * N
    nodes_value = []
    # create nodes
    for i, val in enumerate(values):
        nodes[i] = Node(val, i)
    
    for k in range (0,K):
      nodes_value.append(nodes[k].value)
      for b in range (0,k):
        nodes[k].neighbourhood.add(b)

      for b in range (k,K):
        nodes[k].neighbourhood.add(b)
      
    

    for z in range (k+1,len(nodes)):

      r = find_k_closest(nodes[z].value,nodes_value, K)
      for k, v, p in r:
        nodes[z].neighbourhood.add(k)
      nodes_value.append(nodes[z].value)

    return nodes

### 3.1 Building and testing the graph

In [0]:
navigable_G = build_navigable_graph(sent_vecs_normed, K=10)

In [40]:
search_time = 0
for i, query in enumerate(test_queries):
    start = time.time()
    ans, hops = search_nsw_basic(test_queries_normed[i], navigable_G) 
    finish = time.time()
    search_time += finish - start
    
    print("\nResults for query:", query)
    for k in ans:
        print("\t", facts[k], "sim=", np.dot(test_queries_normed[i], sent_vecs_normed[k]))
        
print("\nSearch in navigable graph took {:.4f} ms on average".format(1000 * (search_time/len(test_queries))))  


Results for query: good mood
	 39. A 2010 study found that 48% of soda fountain contained fecal bacteria, and 11% contained E. Coli. sim= 0.3566344
	 6. There are more lifeforms living on your skin than there are people on the planet. sim= 0.48800442
	 5. You burn more calories sleeping than you do watching television. sim= 0.5034689

Results for query: birds
	 55. The word “gorilla” is derived from a Greek word meaning, “A tribe of hairy women.” sim= 0.46806565
	 47. Avocados are poisonous to birds. sim= 0.7113577

Results for query: virus and bacteria
	 129. In the past 20 years, scientists have found over 1,000 planets outside of our solar system. sim= 0.4419864
	 73. Scientists have tracked butterflies that travel over 3,000 miles. sim= 0.512032
	 47. Avocados are poisonous to birds. sim= 0.6076415

Search in navigable graph took 4.9179 ms on average


## 4. Comparing search quality in resulting graphs

For both graphs, for each data sample, retrieve K nearest neighbours and compare them to the true nearest neighbours of the sample. If the retrieved result is present in the true top-k list of the sample, then it is counted as a hit. For both graphs, report the total number of hits, and the average number of hits per sample.

For example: `Number of hits 394 out of 795, avg per query 2.48`

In [78]:
#TODO measure and report the described metrics
# for i, query in enumerate(test_queries):
hits1 = 0
hits2 = 0
for sent in facts: 
  test_vectors = np.array([model.infer_vector(word_tokenize(sent))])
  test_queries_normed = norm_vectors(test_vectors)
  r = find_k_closest(test_queries_normed, sent_vecs_normed)
  lis = []
  for k, v, p in r:
    lis.append(k)
  ans, hops = search_nsw_basic(test_queries_normed, G)
  navi, hops1 = search_nsw_basic(test_queries_normed, navigable_G) 

  hits1+=len(set(lis).intersection(ans))
  hits2+=len(set(lis).intersection(navi))

print("Number of hits for not navigatable graph",hits1,"out of",len(facts)*5,", avg per query", hits1/len(facts))
print("Number of hits for navigatable graph",hits2,"out of",len(facts)*5,", avg per query",hits2/len(facts))

Number of hits for not navigatable graph 72 out of 795 , avg per query 0.4528301886792453
Number of hits for navigatable graph 122 out of 795 , avg per query 0.7672955974842768


### Bonus task

Generate a small set of 2d points and build 2 types of graphs for this set: small-world graph based on Watts–Strogatz algorithm, and Navigable small-world graph. Visualize both graphs and analyze their structures - do they differ? Does Navigable small-world graph capture geometric proximity better?

In [0]:
#TODO bonus task